# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [4]:
# Data Cleaning
df.duplicated().sum(), df.duplicated(subset=['timestamp']).sum()

(0, 0)

In [5]:
df.isna().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [6]:
df['group'].unique()

array(['experiment', 'control'], dtype=object)

In [7]:
df['action'].unique()

array(['view', 'click'], dtype=object)

In [8]:
# Some EDA
(df['action']=='click').sum()

1860

In [9]:
df['count'] = None
df.fillna(0, inplace=True)
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,0
1,2016-09-24 19:19:03.542569,434745,experiment,view,0
2,2016-09-24 19:36:00.944135,507599,experiment,view,0
3,2016-09-24 19:59:02.646620,671993,control,view,0
4,2016-09-24 20:26:14.466886,536734,experiment,view,0


In [10]:
df.loc[df['action'] == 'click', ['count']] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,0
1,2016-09-24 19:19:03.542569,434745,experiment,view,0
2,2016-09-24 19:36:00.944135,507599,experiment,view,0
3,2016-09-24 19:59:02.646620,671993,control,view,0
4,2016-09-24 20:26:14.466886,536734,experiment,view,0


In [11]:
df.groupby('group').sum()['count']

group
control       932
experiment    928
Name: count, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
#Your code here
control = df.loc[df['group']=='control', ['count']]
experiment = df.loc[df['group']=='experiment', ['count']]
scs.ttest_ind(control, experiment)
# p value is 0.053 which is greater than 0.05. We fail to reject null hypothises
# Therefore, experimental homepage was the same as that of the control group.

Ttest_indResult(statistic=array([-1.93347518]), pvalue=array([0.05321212]))

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [13]:
df.groupby('group').sum()['count']

group
control       932
experiment    928
Name: count, dtype: int64

In [14]:
n_control = (df['group']=='control').sum()
n_experiment = (df['group']=='experiment').sum()
rate_control = 932 / n_control
rate_experiment = 928 / n_experiment
expected_mean = n_experiment*rate_control
expected_mean

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [15]:
#Your code here
std = np.sqrt(expected_mean*(1-rate_control))
std

25.888551746650283

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [16]:
#Your code here
z = (928-expected_mean) / std
p_value = 1 - scs.norm.cdf(z)
p_value

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p value here is 0.0033 which is much less than 0.05. And we can reject the null hypothises here.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.